<a href="https://colab.research.google.com/github/Mallesh06/Smart-Rain-Detection-Automated-Irrigation/blob/main/Rain_Detection_and_Pump_Control_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers torch torchvision datasets pillow


In [ ]:
from transformers import ViTFeatureExtractor, ViTForImageClassification, TrainingArguments, Trainer
from datasets import load_dataset
from torchvision import transforms
from PIL import Image
import torch


In [ ]:
model_name = "google/vit-base-patch16-224-in21k"
feature_extractor = ViTFeatureExtractor.from_pretrained(model_name)
model = ViTForImageClassification.from_pretrained(model_name, num_labels=3) # Changed num_labels to 3

In [ ]:
from datasets import load_dataset

dataset = load_dataset("imagefolder", data_dir="/content/drive/MyDrive/Colab Notebooks/dataset")

# Verify
print(dataset)


In [ ]:
normalize = transforms.Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std)

train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    normalize,
])


In [ ]:
training_args = TrainingArguments(
    output_dir="./vit-finetuned-rain",
    per_device_train_batch_size=8,
    eval_strategy="steps",
    num_train_epochs=3,  # You can increase to 5–10 if you have GPU time
    save_steps=250,
    save_total_limit=2,
    logging_steps=10,
    learning_rate=2e-5,
    push_to_hub=False,
    report_to="none",  # Disable Weights & Biases logging
)

In [ ]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [ ]:
def preprocess_image(examples):
    # Apply the defined transformations to the 'image' column
    examples["pixel_values"] = [train_transforms(image.convert("RGB")) for image in examples["image"]]
    # Return only the processed pixel values and the labels
    return {"pixel_values": examples["pixel_values"], "label": examples["label"]}

# Apply the preprocessing function to the dataset
processed_dataset = dataset.map(preprocess_image, batched=True)

# Remove the original 'image' column as it's no longer needed
processed_dataset = processed_dataset.remove_columns(["image"])

# Verify the structure of the processed dataset
print(processed_dataset)

In [ ]:
unique_labels = set()

for split in processed_dataset:
    for example in processed_dataset[split]:
        unique_labels.add(example['label'])

print("Unique labels found in the dataset:", unique_labels)

In [ ]:
from sklearn.metrics import accuracy_score

def compute_metrics(p):
    preds = p.predictions.argmax(-1)
    return {"accuracy": accuracy_score(p.label_ids, preds)}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=processed_dataset["train"],
    eval_dataset=processed_dataset["test"],
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

In [ ]:
from PIL import Image
import torch

image = Image.open("/content/drive/MyDrive/Colab Notebooks/cloudy6.jpg").convert("RGB")
inputs = feature_extractor(images=image, return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs)
    preds = torch.nn.functional.softmax(outputs.logits, dim=-1)
    label = preds.argmax(-1).item()

labels = ["Rainy", "Not_Rainy"]
print(f"Prediction: {labels[label]}")


In [ ]:
if labels[label] == "Rainy":
    print("💧 Rain detected → Stopping irrigation pump.")
else:
    print("☀️ No rain detected → Pump remains ON.")


In [ ]:
model.save_pretrained("/content/vit-rain-model")


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
model.push_to_hub("MALLESH06/smart-rain-detection")

In [ ]:
model.save_pretrained("/content/vit-rain-model")

In [ ]:
import nbformat

# Load the notebook
with open("/content/drive/MyDrive/Colab Notebooks/Smart Rain Detection and Pump Control System.ipynb", "r") as f:
    notebook = nbformat.read(f, as_version=4)

# Remove the 'widgets' key from the metadata if it exists
if 'widgets' in notebook.metadata:
    del notebook.metadata['widgets']

# Save the modified notebook
with open("/content/drive/MyDrive/Colab Notebooks/Smart Rain Detection and Pump Control System.ipynb", "w") as f:
    nbformat.write(notebook, f)

print("Notebook metadata updated. You can now try pushing to GitHub again.")

In [ ]:
!git clone https://github.com/Mallesh06/Smart-Rain-Detection-Automated-Irrigation.git
!ls Smart-Rain-Detection-Automated-Irrigation/*.ipynb